In [ ]:
%pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("")
df

In [ ]:
df.plot(kind='scatter', x='Sales_Amount', y='Quantity_Sold', color='red');

In [ ]:
from sklearn.model_selection import linear_model

modelo = linear_model.LinearRegression()